<a href="https://colab.research.google.com/github/ckim415/OOOA/blob/main/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_by_%ED%83%80%EC%9D%B4%ED%8B%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 코렙에서 한글 사용

In [1]:
!apt -qq -y install fonts-nanum
 
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
 
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10)
fm._rebuild()
 
# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'
 
# Colab 의 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic') 

fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


# 구글드라이브 연동

In [2]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

# 구글 드라이브 파일 확인
!ls '/gdrive/My Drive/프로젝트2'

# 반복되는 드라이브 경로 변수화
drive_path = '/gdrive/My Drive/프로젝트2/'

Mounted at /gdrive
 6144_link.csv		       edu_link_df
'추천시스템 by 타이틀.ipynb'   game_df.csv
 cate_keyword.csv	       home.csv
 concat_df.csv		      '프로젝트 데이터 분석.ipynb'
 crawling_final.ipynb	       Untitled2.ipynb
 df_nouns_add.csv


In [3]:
import pandas as pd
import numpy as np

# 데이터 가져오기

In [4]:
df = pd.read_csv(drive_path + 'concat_df.csv')
df.head()

,Unnamed: 0,categories,title,company_names,target_amounts,funding_amounts,percentages,n_supporters,likes,durations,start,end,time,descriptions,result
0,0,뷰티,[와디즈 최초]20대 딸이랑 50대 엄마랑 함께 쓰는 주름개선 만능 크림,주식회사칼라코드,500000,2338880,467,54,41,"['2020.02.28', '2020.03.06']",2020-02-28,2020-03-06,7,"테라젠의 첫 번째 제품, 20대 딸이랑 50대 엄마가 함께 쓰는 주름개선 만능 크림...",1
1,1,뷰티,[2.8억 앵콜] 하루만에 수분140%증가! 임상검증 완료! 갈바닉마스크,반디비타 주식회사,1000000,14260000,1426,145,79,"['2021.04.26', '2021.05.31']",2021-04-26,2021-05-31,35,지금 쓰는 스킨케어 제품의 수분증발을 잡아주고 유효성분은 갈바닉으로 피부 깊숙히 전...,1
2,2,뷰티,1000개 유명 피부과에 공급되는 시크릿 포뮬러를 담은 33패드 프로그램,(주)코떼랑,500000,5017500,1003,65,65,"['2019.04.16', '2019.05.06']",2019-04-16,2019-05-06,20,위시포뮬러가 피부과 화장품 제조 20년의 기술과 노하우를 단 한 장의 패드에 담았...,1
3,3,뷰티,두피 관리의 핵심은 유분량의 균형입니다! 올솔 모닝-나이트 밸런스 샴푸,올솔 (allsol),500000,7358800,1471,134,130,"['2020.03.13', '2020.04.06']",2020-03-13,2020-04-06,24,매일 변하는 두피 유분량! \n그런데도 매번 같은 타입의 샴푸 쓰세요? \n두피뿐만...,1
4,4,뷰티,컨디셔너 필요없이 그린빈샴푸 하나로 두피와 모발의 건강을 책임집니다.,주라이프 네츄럴 코스메틱 주앤솝,500000,694800,138,30,12,"['2019.04.26', '2019.05.08']",2019-04-26,2019-05-08,12,"화학 성분 NO! 모발에 윤기와 탄력을 주며, 비듬이나 가려움을 덜어주며, 컨디셔너...",1


# text 처리

*   특수문자 제거




In [5]:
import re
from string import punctuation

In [6]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [7]:
!pip install konlpy wordcloud

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 6.8MB/s 
     |████████████████████████████████| 460kB 47.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [8]:
from konlpy.tag import Kkma, Hannanum, Komoran, Mecab, Okt

In [9]:
kkma = Kkma()
han = Hannanum()
kor = Komoran()
okt = Okt()
# mecab = Mecab()

In [10]:
#punctutation 제거 패턴
pattern1 = re.compile(r'[{}]'.format(re.escape(punctuation)))
#숫자 제거 패턴
pattern2 = re.compile(r'[0-9]')

In [15]:
# 함수화
def cleaning(doc):

  sentence = pattern1.sub(' ',
             pattern2.sub('',str(doc))).strip()

  return sentence

In [ ]:
for i in df['title'][0:30]:
  print(i)
  sentence = pattern1.sub(' ',
             pattern2.sub('',i)).strip()
  print('kkma :',kkma.nouns(sentence))
  print('han :',han.nouns(sentence))
  print('okt :',okt.nouns(sentence))
  print('komoran :',kor.nouns(sentence))
  #print('mecab :',Mecab.nouns(sentence))

  print('-'*10)

In [21]:
df['title_clean'] = df['title'].apply(cleaning)
df['title_clean']

0             와디즈 최초 대 딸이랑 대 엄마랑 함께 쓰는 주름개선 만능 크림
1               억 앵콜  하루만에 수분 증가  임상검증 완료  갈바닉마스크
2              개 유명 피부과에 공급되는 시크릿 포뮬러를 담은 패드 프로그램
3         두피 관리의 핵심은 유분량의 균형입니다  올솔 모닝 나이트 밸런스 샴푸
4           컨디셔너 필요없이 그린빈샴푸 하나로 두피와 모발의 건강을 책임집니다
                           ...                   
21475            히트패드  출근길의 모든 핸들을 뜨겁게  붙이는 전기 핫팩
21476    SuelMu 무선 에어펌프 자동차 오토바이공기압  자전거 튜브 축구공까지
21477          너도  나도 간단하게 개국어지원 반응형 웹페이지 제작 실크로드
21478                         웨어러블 공기청정기  전동식 마스크
21479                                   년JD MJD시계
Name: title_clean, Length: 21480, dtype: object

## 부호 제거된 title
- noun 추출
- 키워드 추출

In [27]:
!pip install krwordrank

In [28]:
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize

In [44]:
def keyword(sentence):
  keywords = normalize(str(sentence), english=True, number=False).split(' ')
  for kw in keywords:
    if len(kw) == 1:
      keywords.remove(kw)
  return keywords

In [34]:
def noun(sentence):
  kkma = Kkma()
  nouns = kkma.nouns(sentence)
  for noun in nouns:
    if len(noun) == 1:
      nouns.remove(noun)
  return nouns

In [35]:
df['nouns'] = df['title_clean'].apply(noun)

In [47]:
df['nouns'][:20]

0            [와디, 와디즈, 최초, 딸, 엄마, 주름, 주름개선, 개선, 만능, 크림]
1     [앵, 앵콜, 하루, 수분, 증가, 임상, 임상검증, 검증, 완료, 바닉마스크, 마스크]
2                 [유명, 피부과, 공급, 시크, 시크릿, 포뮬러, 패드, 프로그램]
3           [두피, 관리, 핵심, 유분량, 분량, 균형, 모닝, 나이트, 밸런스, 샴푸]
4                        [컨디셔너, 필요, 샴푸, 하나, 두피, 모발, 건강]
5          [후기, 후기폭주, 폭주, 만개, 판매, 덕후, 리얼, 핸드, 핸드크림, 크림]
6                        [비건인증콜라겐, 건인, 콜라겐, 방울, 피부, 처방]
7                               [아침, 피부, 실화, 마스크, 마스크팩]
8             [손안, 에스테, 에스테틱, 초당, 진동, 마법, 매직, 매직스틱, 스틱]
9                                     [목, 대용량, 키웨이, 웨이]
10          [앵콜, 명품, 명품화장품년경력, 화장품, 경력, 선, 선크림, 크림, 피부]
11                                 [건조성, 피부, 프리미엄, 에센스]
12                                       [탈모, 탈모관리, 관리]
13              [편안, 기분, 잠자리, 아로마, 아로마테라피, 테라, 스위트, 드림]
14           [한번, 머리, 피부, 어디, 민감성, 민감성피부, 지성, 지성피부, 누구]
15    [피부, 피부관리, 관리, 클렌징, ㅣ하나, 하나, 끝낸다ㅣ스피, 미라, 듀얼, 퍼...
16                           [앵콜, 후기, 입증, 저분자, 루론, 흡수력]
17           [유해, 물질, 아이, 피부, 습관, 만들기, 유아, 유아클렌징

In [45]:
df['keywords'] = df['title_clean'].apply(keyword)

In [46]:
df['keywords'][:10]

0            [와디즈, 최초, 딸이랑, 엄마랑, 함께, 쓰는, 주름개선, 만능, 크림]
1                 [앵콜, 하루만에, 수분, 증가, 임상검증, 완료, 갈바닉마스크]
2            [유명, 피부과에, 공급되는, 시크릿, 포뮬러를, 담은, 패드, 프로그램]
3    [두피, 관리의, 핵심은, 유분량의, 균형입니다, 올솔, 모닝, 나이트, 밸런스, 샴푸]
4       [컨디셔너, 필요없이, 그린빈샴푸, 하나로, 두피와, 모발의, 건강을, 책임집니다]
5        [후기폭주, 만개, 판매, 자몽, 덕후가, 만든, 리얼, 자몽, 퍼퓸, 핸드크림]
6                  [비건인증콜라겐, 매일, 방울, 탄탄한, 피부를, 위한, 처방]
7                    [아침마다, 달라지는, 피부, 실화야, 뽑아쓰는, 마스크팩]
8                   [손안에, 작은, 에스테틱, 초당, 진동의, 마법, 매직스틱]
9            [세럼은, 모두, 작고, 비쌀까, 목에도, 바르는, 대용량, 밀키웨이세럼]
Name: keywords, dtype: object

## 파일 저장

In [48]:
df.to_csv(drive_path+'df_added.csv')

In [51]:
check=pd.DataFrame(df['nouns'] == df['keywords'])
check

,0
0,False
1,False
2,False
3,False
4,False
...,...
21475,False
21476,False
21477,False
21478,False


In [52]:
check[0].value_counts()

False    21435
True        45
Name: 0, dtype: int64

## check
- if keywords == nouns 
- False = 21435
- True = 45

#추천 시스템 by item / df['nouns']

In [53]:
df[['categories','nouns']]

,categories,nouns
0,뷰티,"[와디, 와디즈, 최초, 딸, 엄마, 주름, 주름개선, 개선, 만능, 크림]"
1,뷰티,"[앵, 앵콜, 하루, 수분, 증가, 임상, 임상검증, 검증, 완료, 바닉마스크, 마스크]"
2,뷰티,"[유명, 피부과, 공급, 시크, 시크릿, 포뮬러, 패드, 프로그램]"
3,뷰티,"[두피, 관리, 핵심, 유분량, 분량, 균형, 모닝, 나이트, 밸런스, 샴푸]"
4,뷰티,"[컨디셔너, 필요, 샴푸, 하나, 두피, 모발, 건강]"
...,...,...
21475,테크·가전,"[히트, 히트패드, 패드, 출근길, 핸들, 전기, 핫팩]"
21476,테크·가전,"[무선, 에어, 에어펌프, 펌프, 자동차, 오토바이, 오토바이공기압, 공기압, 자전..."
21477,테크·가전,"[개, 개국어지원, 국어, 지원, 반응, 반응형, 웹, 웹페이지, 페이지, 제작, ..."
21478,테크·가전,"[웨어, 웨어러, 공기, 공기청정기, 청정기, 전동, 마스크]"


In [54]:
# 단어 리스트를 문자열로 변환
df['nouns_literal'] = df['nouns'].apply( lambda x : (' ').join(x))

In [56]:
df['nouns_literal'][:10]

0          와디 와디즈 최초 딸 엄마 주름 주름개선 개선 만능 크림
1    앵 앵콜 하루 수분 증가 임상 임상검증 검증 완료 바닉마스크 마스크
2             유명 피부과 공급 시크 시크릿 포뮬러 패드 프로그램
3         두피 관리 핵심 유분량 분량 균형 모닝 나이트 밸런스 샴푸
4                   컨디셔너 필요 샴푸 하나 두피 모발 건강
5        후기 후기폭주 폭주 만개 판매 덕후 리얼 핸드 핸드크림 크림
6                  비건인증콜라겐 건인 콜라겐 방울 피부 처방
7                        아침 피부 실화 마스크 마스크팩
8          손안 에스테 에스테틱 초당 진동 마법 매직 매직스틱 스틱
9                             목 대용량 키웨이 웨이
Name: nouns_literal, dtype: object

## 테이블 저장

In [105]:
df.to_csv(drive_path+'df_reco_system.csv')

## 추천시스템

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [59]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))           
nouns_mat = count_vect.fit_transform( df['nouns_literal'] )
nouns_mat.shape

(21480, 126771)

In [60]:
# 코사인 유사도 계산계산
# 반환된 코사인 유사도 행렬의 크기 및 앞 2개 데이터만 추출

nouns_sim = cosine_similarity(nouns_mat, nouns_mat)
nouns_sim.shape

(21480, 21480)

In [61]:
# 유사도가 높은 순으로 정리된 genre_sim 객체의 비교 행 위치 인덱스 값
# 값이 높은 순으로 정렬된 비교 대상 행의 유사도 값이 아니라
#  비교 대상 행의 위치 인덱스임에 주의
nouns_sim_sorded_ind = nouns_sim.argsort()[:, ::-1]
nouns_sim_sorded_ind[:1]

array([[    0,  1611,   559, ..., 14230, 14231, 10739]])

In [ ]:
def find_sim_title(df, sorted_ind, noun, top_n=10):
  title_movie = df[ df['title'].split() == title_name ]
  title_index = title_movie.index.values
  sim_indexs = sorted_ind[title_index, :(top_n)]
  sim_indexs = sim_indexs.reshape(-1)
  return df.iloc[sim_indexs]

# 카테고리별 keyword 추출

### *** title마다 keyword추출을 불가능*** 

In [ ]:
df['categories'].value_counts()

패션·잡화       5208
푸드          3066
홈리빙         2949
뷰티          2484
테크·가전       2124
반려동물        1635
소셜·캠페인       832
스포츠·모빌리티     694
여행·레저        674
교육·키즈        666
공연·컬쳐        564
출판           469
디자인소품         48
기부·후원         41
모임            26
Name: categories, dtype: int64

In [ ]:
categories = df['categories'].value_counts().index
categories

Index(['패션·잡화', '푸드', '홈리빙', '뷰티', '테크·가전', '반려동물', '소셜·캠페인', '스포츠·모빌리티',
       '여행·레저', '교육·키즈', '공연·컬쳐', '출판', '디자인소품', '기부·후원', '모임'],
      dtype='object')

In [ ]:
#기부후원만 test
titles = df[df['categories'] == '기부·후원']['title'].values
normalized_titles = [normalize(title, english=True, number=True) for title in titles]

In [ ]:
normalized_titles

In [ ]:
# keyword 추출
wordrank_extractor = KRWordRank(
      min_count = 2, # 단어의 최소 출현 빈도수 (그래프 생성 시)
      max_length = 5, # 단어의 최대 길이
      verbose = True
      )
  
beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

keywords, rank, graph = wordrank_extractor.extract(normalized_titles, beta, max_iter) 

scan vocabs ... 
num vocabs = 168
done


In [ ]:
keywords.keys()

dict_keys(['위한', '청소년', '세월호', '네팔', '긴급구호', '있도록', '준비합니다', '의인', '따뜻한', '꿈을', '보내', '희망', '대학언론', '선물', '작은', '새로', '피해', '문화', '우리', '위안', '아이', '리더십', '응원', 'ST', '친구'])

In [ ]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
    print('%8s:\t%.4f' % (word, r))

      위한:	3.0330
     청소년:	2.4551
     세월호:	1.9163
      네팔:	1.8873
    긴급구호:	1.7646
     있도록:	1.5974
   준비합니다:	1.5753
      의인:	1.4829
     따뜻한:	1.4332
      꿈을:	1.3289
      보내:	1.3151
      희망:	1.3052
    대학언론:	1.1182
      선물:	1.0348
      작은:	1.0000
      새로:	1.0000
      피해:	0.9943
      문화:	0.9574
      우리:	0.8836
      위안:	0.8149
      아이:	0.7888
     리더십:	0.7063
      응원:	0.5154
      ST:	0.4908
      친구:	0.4306


In [ ]:
#한꺼번에 키워드 추출 카테고리 별로 키워드 추출/ 'title_noun'을 사용
beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10
total = {}
categoris = df['categories'].value_counts()
for cate in categoris.index:
  titles = df[df['categories'] == cate]['title_noun'].values
  texts = [normalize(text, english=True, number=True) for text in titles]
  wordrank_extractor = KRWordRank(
    min_count = 5, # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 10, # 단어의 최대 길이
    verbose = True
    )



  keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)
  # for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
  total[cate] = ['%s' % (word) for word,r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]]
  

scan vocabs ... 
num vocabs = 3766
done
scan vocabs ... 
num vocabs = 2878
done
scan vocabs ... 
num vocabs = 2744
done
scan vocabs ... 
num vocabs = 2249
done
scan vocabs ... 
num vocabs = 2196
done
scan vocabs ... 
num vocabs = 1590
done
scan vocabs ... 
num vocabs = 847
done
scan vocabs ... 
num vocabs = 989
done
scan vocabs ... 
num vocabs = 862
done
scan vocabs ... 
num vocabs = 831
done
scan vocabs ... 
num vocabs = 730
done
scan vocabs ... 
num vocabs = 615
done
scan vocabs ... 
num vocabs = 64
done
scan vocabs ... 
num vocabs = 15
done
scan vocabs ... 
num vocabs = 20
done


In [ ]:
for cate in categoris.index:
  print(cate, total[cate],'\n')

패션·잡화 ['만원', '천연', '가방', '다이', '가죽', '데일리', '안경', '명품', '스타', '자켓', '하나', '수납', '디자', '편안', '지갑', '코트', '셔츠', '아몬드', '당신', '인생', '미니', '케이스', '가격', '핸드', '마스크', '아이', '스니커즈', '일상', '끝판', '후드'] 

푸드 ['선물', '건강', '프로', '커피', '프리', '제주', '인생', '단백질', '추석', '하루', '설탕', '세트', '끝판', '밀크', '수제', '바이오', '한우', '갈비', '국내', '보리', '초콜릿', '아이', '식사', '한정', '에너지', '비타민', '전통', '발효', '참치', '진짜'] 

홈리빙 ['베개', '만원', '주방', '공기', '매트', '공간', '천연', '핸드', '항균', '원목', '프로젝트', '감성', '미세', '테이블', '당신', '우리', '건강', '샤워', '커피', '수면', '바디', '아이', '하나', '섬유', '끝판', '디자인', '쿠션', '선물', '호텔', '생활'] 

뷰티 ['피부', '크림', '케어', '마스크', '성분', '바디', '스킨', '클렌징', '수분', '관리', '천연', '화장', '보습', '앰플', '모공', '미백', '샴푸', '향수', '두피', '탈모', '핸드', '헤어', '주름', '진정', '각질', '특허', '여성', '임상', '프로', '줄기'] 

테크·가전 ['무선', '스마트', '충전', '공기', '청정기', '살균', '만원', '고속', '휴대용', '끝판', '에어', '보조', '아이', '이어폰', '필수', '하나', '거치대', '청소기', '패드', '배터리', '먼지', '글로벌', '스피커', '미세', '칫솔', '블루', '전기', '전동', '차량', '국내'] 

반려동물 ['반려', '동물', '고양이', '강아지', '건강',

## save dict total to csv file

In [ ]:
import csv
import sys
with open(drive_path+'cate_keyword.csv','w') as f:
    w = csv.DictWriter(f, total.keys())
    w.writeheader()
    w.writerow(total)

In [ ]:
#명사만 먼저 추출해서 키워드 추출 가능한지 (카테고리 별로 ) 확인 요망

## 확인 결과 : 기부 후원 빼고 가능함

# 카테고리별 펀딩 평균 모금액 

In [ ]:
df['start_year'] = pd.to_datetime(df['start']).dt.to_period('Y')

df_year_mean = df.pivot_table(columns='categories', index='start_year', 
                                      values='funding_amounts',aggfunc=np.mean)

In [ ]:
df_year_mean= df_year_mean.fillna(0) 

In [ ]:
df_year_mean = df_year_mean.astype('int')

In [ ]:
df_year_mean.T

start_year,2013,2014,2015,2016,2017,2018,2019,2020,2021
categories,,,,,,,,,
공연·컬쳐,1577900,2875927,1608609,3314067,5216102,4880764,6086437,18670049,90022098
교육·키즈,0,16857750,5986921,3470807,9611619,6791423,6873404,9203267,13615529
기부·후원,0,0,2025436,369282,0,2129178,535000,613875,0
디자인소품,0,0,0,0,555297703,141703127,34881690,35242113,2863712
모임,0,0,0,0,0,0,2380300,1946200,0
반려동물,0,0,2222266,3501283,8863727,17166607,10673292,9172378,8350958
뷰티,0,0,775375,3283598,6108298,8186882,9732600,11535084,14443909
소셜·캠페인,4425204,2054491,3910569,2529761,3148786,3581705,4552107,26170944,19286872
스포츠·모빌리티,155000,2822400,8135625,33204902,42857413,22589399,25101891,30249591,26294317


In [ ]:
cult=df[df['categories'] == '공연·컬쳐'][['funding_amounts','start_year']]#['funding_amounts'].mean()

In [ ]:
cult[cult['start_year']=='2013']['funding_amounts']

2568    1540000
2616    1060000
2617    3525000
2694     305000
2706     750000
2713     220000
2741    2235000
2773    1550000
2844    3060000
3000    1534000
Name: funding_amounts, dtype: int64

In [ ]:
cult[cult['start_year']=='2013']['funding_amounts'].mean()

1577900.0